In [3]:
import torch #pytorch library that helps in building the deep leanring algorithms
from torch import nn ##nn means neural network
from torch.utils.data import DataLoader #performs the process of batching
from torchvision import datasets #Inbuild dataset that pytorch library has - FashionMnist Dataset
from torchvision.transforms import ToTensor

In [4]:
training_data = datasets.FashionMNIST(root="data",train=True,download=True,transform=ToTensor())
test_data = datasets.FashionMNIST(root="data",train=False,download=True,transform=ToTensor())

In [5]:
training_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [6]:
test_data

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

**Batching of this data**





In [7]:
batch_size = 64

train_dataloader = DataLoader(training_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)

In [8]:

for X,y in test_dataloader: #Image - Color image shape (batch_size,number of channel,length,width)
  print(X.shape)            #Image - Black and white image - number of channels is 1
  print(y.shape)
  break

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [9]:

device = "cuda" if torch.cuda.is_available() else "cpu" #torch.cuda.is_available() checks for your system has gpu or cpu
device

'cuda'

In [10]:
class NeuralNetwork(nn.Module): #Child class and nn.Module is a parent class -- (defined in the pytorch library)

  def __init__(self): #declare the architecture
    super().__init__() #basically initailizes all the variables of the parent class
    self.flatten = nn.Flatten() #28x28 image into a 764x1 vector
    self.linear1 = nn.Linear(28*28,512)
    self.linear2 = nn.Linear(512,512)
    self.linear3 = nn.Linear(512,10)
    self.relu = nn.ReLU()
  def forward(self,x):#is always used to pass the inputs to the neural network
    x = self.flatten(x)
    x = self.linear1(x)
    x = self.relu(x)
    x = self.linear2(x)
    x = self.relu(x)
    x = self.linear3(x)
    return x

In [11]:
model = NeuralNetwork()
model = model.to(device) #copies your entire architecture to the GPU

In [12]:
loss_fn = nn.CrossEntropyLoss() #cross entropy loss
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3) #stochastic Gradient descent

In [13]:
#steps in the GD : Batch of the input / Pass it to the model / Compute loss function / Update the weights

def train(dataloader,model,loss_fn,optimizer):
  model.train() #putting the model in the training mode
  for batch,(X,y) in enumerate(dataloader):
    #sending the data to the GPU
    X = X.to(device)
    y = y.to(device)

    #Compute predictions
    pred = model(X)

    #Compute loss
    loss = loss_fn(pred,y)

    #Backpropogation
    loss.backward() #Wnew = Wold - lr*dl/dw
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      print(f'Loss of the Model {loss.item()}')


In [14]:
def test(dataloader,model,loss_fn):
  model.eval() #putting the model in the training mode
  num_batched = len(dataloader)
  test_loss, correct = 0,0
  with torch.no_grad(): #We will not compute gradients for the test data
    for X,y in dataloader:
      X = X.to(device)
      y = y.to(device)

      #Compute predictions
      pred = model(X)

      #Compute loss
      test_loss += loss_fn(pred,y).item()

      #Find how many correct predictions
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss = test_loss/num_batched
  correct = correct/(len(dataloader.dataset))

  print(f'Test Accuracy {100*correct}, Avg_loss : {test_loss}')


In [15]:
epochs = 5

for t in range(5):
  print(f'Epoch {t+1}')
  train(train_dataloader,model,loss_fn,optimizer)
  test(test_dataloader,model,loss_fn)


Epoch 1
Loss of the Model 2.302501916885376
Loss of the Model 2.2908520698547363
Loss of the Model 2.2748072147369385
Loss of the Model 2.2680413722991943
Loss of the Model 2.2573087215423584
Loss of the Model 2.235004186630249
Loss of the Model 2.239778995513916
Loss of the Model 2.2135329246520996
Loss of the Model 2.213132381439209
Loss of the Model 2.1839537620544434
Test Accuracy 50.42, Avg_loss : 2.1767884682697853
Epoch 2
Loss of the Model 2.188882350921631
Loss of the Model 2.183500289916992
Loss of the Model 2.1307737827301025
Loss of the Model 2.142164707183838
Loss of the Model 2.1095199584960938
Loss of the Model 2.051478862762451
Loss of the Model 2.0818376541137695
Loss of the Model 2.014612913131714
Loss of the Model 2.021108865737915
Loss of the Model 1.955333948135376
Test Accuracy 58.830000000000005, Avg_loss : 1.9535347542185693
Epoch 3
Loss of the Model 1.979581594467163
Loss of the Model 1.9650651216506958
Loss of the Model 1.8521820306777954
Loss of the Model 1.89

In [16]:
torch.save(model.state_dict(),"/content/iteration1.pth")

In [17]:

model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("/content/iteration1.pth"))

<All keys matched successfully>

In [18]:

classes = ["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle Boot"]

model.eval()
X,y = test_data[0][0], test_data[0][1]

with torch.no_grad():
  X = X.to(device)
  pred = model(X)
  predicted,actual = classes[pred[0].argmax(0)],classes[y]
  print(f'Predicted {predicted}')
  print(f'Actual {actual}')

Predicted Ankle Boot
Actual Ankle Boot


In [19]:
y

9